In [ ]:
%cd ~/comp_mut

In [ ]:
%reset -f

In [ ]:
import json
from collections import defaultdict, Counter
import argparse
import os
from tqdm import tqdm
import sys
import csv
import pathogenprofiler as pp
import tbprofiler
from csv import DictReader
from collections import Counter
import requests
from contextlib import closing
import re
from python_scripts.utils import *

In [ ]:

def get_vars_exclude(vars_exclude_file):

    # URL below is the results of all Fst = 1 variants from https://genomemedicine.biomedcentral.com/articles/10.1186/s13073-020-00817-3
    fst_results_url = 'https://raw.githubusercontent.com/GaryNapier/tb-lineages/main/fst_results_clean_fst_1_for_paper.csv'
    # See https://www.codegrepper.com/code-examples/python/how+to+read+a+csv+file+from+a+url+with+python for pulling data from url
    with closing(requests.get(fst_results_url, stream=True)) as r:
        f = (line.decode('utf-8') for line in r.iter_lines())
        fst_dict = csv_to_dict_multi(f)
    
    lin_specific_variants = []
    for gene in fst_dict:
#         if gene in genes:
        for var in fst_dict[gene]:
            lin_specific_variants.append( tuple( [ gene, reformat_mutations(var['aa_pos']) ] ) )

    # Read in variants to be excluded
    vars_exclude = []
    for l in open(vars_exclude_file):
        vars_exclude.append(tuple(l.strip().split(',')))

    # Concat
    vars_exclude = vars_exclude + lin_specific_variants 
    return vars_exclude

def get_counts(meta,samples,col):
    return dict(Counter([meta[s][col] for s in samples]))

def get_meta_proportion(meta,samples,column,targets):
    tmp = get_counts(meta,samples,column)
    target_count = sum([tmp.get(c,0) for c in targets])
    return round(target_count/sum(tmp.values()), 3)

def filter_vars(variants, mutation2sample, meta_dict, drug_of_interest):

    # Get stats for each variant
    stats_dict = defaultdict(dict)
    variants_passed = set()
    # for var in variants:
    for var in variants:

        # Get proportion or number of samples (in the case of lineage) per potential mutation
        samps = mutation2sample[var]

        if len(samps)<3: continue

        dst_proportion = get_meta_proportion(meta_dict,samps,drug_of_interest,['1'])
        sensitive_geno_proportion = get_meta_proportion(meta_dict,samps,'drtype',['Sensitive'])
        num_lins = len(set(resolve_lineages(get_counts(meta_dict,samps,'sublin'))))

        # Filter and add to dict
        if dst_proportion >= 0.5 and sensitive_geno_proportion <= 0.5 and num_lins > 1:
            variants_passed.add(var)
            stats_dict[var] = {'var': var, 
                               'gene': var[0], 
                               'mutation': var[1],
                               'n_samps' : len(samps), 
                               'dst_prop': dst_proportion, 
                               'dr_prop': sensitive_geno_proportion, 
                               'n_lins': num_lins}

    return (variants_passed, stats_dict)


In [ ]:
drug_of_interest = 'rifampicin'

In [ ]:
# FILES

# potential_comp_mut_file = args.potential_comp_mut_file
# metadata_file = args.metadata_file
# tbdb_file = args.tbdb_file
# drtypes_file = args.drtypes_file
# known_comp_mut_file = args.known_comp_mut_file
# tbprofiler_results_dir = args.tbprofiler_results_dir
# vars_exclude_file = args.vars_exclude_file
# potential_res_mut_outfile = args.potential_res_mut_outfile

# FILES

potential_comp_mut_file = "results/%s_novel_comp_mut_model_results.csv" % drug_of_interest
metadata_file = "../metadata/tb_data_18_02_2021.csv"
tbdb_file = "../tbdb/tbdb.csv"
drtypes_file = "../pipeline/db/dr_types.json"
known_comp_mut_file = '../pipeline/db/compensatory_mutations.csv'
tbprofiler_results_dir = '/mnt/storage7/jody/tb_ena/tbprofiler/freebayes/results/'
vars_exclude_file = 'metadata/var_exclude_comp_mut.csv'
# potential_res_mut_outfile = 'results/isoniazid_potential_res_mut_stats.csv'


In [ ]:
# VARIABLES

# suffix = args.suffix
# drug_of_interest = args.drug_of_interest

suffix = ".results.json"

In [ ]:
# Get known compensatory mutations for drug of interest
compensatory_mutations = defaultdict(set)
for row in csv.DictReader(open(known_comp_mut_file)):
    if row['Drug'] != drug_of_interest: continue
    compensatory_mutations[row['Drug']].add((row['Gene'],row['Mutation']))
compensatory_mutations

In [ ]:
len(compensatory_mutations[drug_of_interest])

In [ ]:
# Read in GLM results file
with open(potential_comp_mut_file, 'r') as f:
    potential_comp_mut_dict = csv_to_dict(f)

potential_comp_mut_dict

In [ ]:
# Convert to list of tuples
potential_comp_mut_list = [(potential_comp_mut_dict[var]['gene'], potential_comp_mut_dict[var]['term'])\
 for var in list(potential_comp_mut_dict)]
potential_comp_mut_list

In [ ]:
# Read in metadata
with open(metadata_file) as mf:
    meta_dict = csv_to_dict(mf)
    
# Pull samples
samples = list(meta_dict.keys())

In [ ]:
# Read in tbdb file
with open(tbdb_file, 'r') as f:
    tbdb_dict = csv_to_dict_multi(f, 'Drug')
    
tbdb_dict[drug_of_interest]

In [ ]:
# Read in DR types from json
standardise_drtype = json.load(open(drtypes_file))

In [ ]:
# Read in variants to exclude
vars_exclude = get_vars_exclude(vars_exclude_file)
vars_exclude

In [ ]:
# WRANGLE DATA

# Find resistance genes associated with drug of interest
# genes = set([l.strip().split()[0] for l in open(genes_file)])
genes = set()
for var in tbdb_dict[drug_of_interest]:
    genes.add(var['Gene'])

# Concat with genes from known and potential resistance mutations
genes = genes.union(set([var[0] for var in compensatory_mutations[drug_of_interest]]))
genes = genes.union(set([var[0] for var in potential_comp_mut_list]))

genes

In [ ]:
# Read in all the json data for (samples with) mutations in drug-of-interest genes only (>0.7 freq) and the metadata for those samples

# Load mutation data using ('gene','change') as keys
mutation2sample = defaultdict(set)
sample2mutation = defaultdict(set)
resistance_mutations = defaultdict(set)
for s in tqdm(samples):
    file = "%s/%s%s" % (tbprofiler_results_dir, s, suffix)
    if os.path.isfile(file):
        data = json.load(open(file))
        # Skip mixed samps
        if ';' in data['sublin']: continue
        
        # Update metadata
        meta_dict[s]['drtype'] = data['drtype']
        meta_dict[s]['sublin'] = data['sublin']

        # MAKE SURE THE FOR LOOP BELOW IS INDENTED IN LINE WITH if os.path.isfile(file):
        # Otherwise adds sample s to mutation2sample etc

        for var in data['dr_variants'] + data['other_variants']:
            if var['gene'] not in genes: continue
            if var['freq'] < 0.7: continue
            if var['type']=='synonymous_variant': continue
            if (var['gene'], var['change']) in vars_exclude: continue
            key = (var['gene'],var['change'])
            mutation2sample[key].add(s)
            sample2mutation[s].add(key)
            if "drugs" in var:
                for d in var["drugs"]:
                    if d["drug"] not in drug_of_interest: continue
                    if key in compensatory_mutations[d["drug"]] or key in potential_comp_mut_list: continue
                    resistance_mutations[d["drug"]].add(key)


In [ ]:
# Classify potential compensatory mutations and filter 
# GLM models (filter_novel_comp_mut.R) is only first step in identifying 'interesting' compensatory mutations
# Need to check against tbprofiler results for each mutation 
# e.g. if the mutation is lineage specific, then filter out
potential_comp_mut_filtered, potential_comp_mut_stats = filter_vars(potential_comp_mut_list, mutation2sample, meta_dict, drug_of_interest)
potential_comp_mut_filtered

In [ ]:
mutation2sample.keys()

In [ ]:
len(sample2mutation.keys())

In [ ]:
resistance_mutations

In [ ]:
potential_comp_mut_stats

In [ ]:
# Add the filtered potential compensatory mutations 
# to the list of known compensatory mutations for the drug of interest
compensatory_mutations[drug_of_interest].update(potential_comp_mut_filtered)

In [ ]:
# ** MAIN BIT **
# ** Go over all the samples and get the potential resistance mutations from the presence of comp mutations ** 
potential_resistance_mutations = set()
for s in tqdm(samples):
#     # Get the comp, res and other variants for each sample in the full sample list
    comp_var = [var for var in sample2mutation[s] if var in compensatory_mutations[drug_of_interest]]
    res_var = [var for var in sample2mutation[s] if var in resistance_mutations[drug_of_interest]]
    other_vars = [var for var in sample2mutation[s] if var not in compensatory_mutations[drug_of_interest]\
                  and var not in resistance_mutations[drug_of_interest]]

    # If there is at least one comp variant and there are no (known) resistance variants
    if len(comp_var)>0 and len(res_var)==0:
        # If there are no 'other' variants print the sample and the comp variants
        if len(other_vars)==0:
            print("Sample with at least one comp. mut. but no res. mutations:")
            print("samp:", s, "comp. mut.:", comp_var)
        # Store the 'other' vars as potential resistance variants
        for var in other_vars:
            potential_resistance_mutations.add(var)


In [ ]:
potential_resistance_mutations

In [ ]:
# Filter the potential resistance variants in the same way as filtering the potential comp. variants
potential_res_mut_filtered, potential_res_mut_stats = filter_vars(potential_resistance_mutations, mutation2sample, meta_dict, drug_of_interest)

In [ ]:
potential_res_mut_stats

In [ ]:
# Write stats dictionary to file:
with open(potential_res_mut_outfile, 'w') as f:
    writer = csv.DictWriter(f, fieldnames = list(get_embedded_keys(potential_res_mut_stats)))
    writer.writeheader()
    for row in potential_res_mut_stats:
        writer.writerow(potential_res_mut_stats[row])